## 코드 추상화: 클래스와 객체 2부

안내: [python-textbook.readthedocs.io의 Classes](https://python-textbok.readthedocs.io/en/1.0/Classes.html#class-attributes) 내용을 요약 및 수정한 내용입니다.

## 클래스 장식자

**장식자**(decorator)는 다른 함수의 기능에 다른 기능을 추가할 때 사용되는 함수이다.
즉, 장식자는 함수를 인자로 받아 그 함수가 하는 일에 더해 다른 기능도
수행하는 함수를 리턴값으로 내준다.
이런 장식자를 함수로 정의할 수 있는 이유는 함수가 제1종 객체이기 때문이다.
즉, 다른 함수의 인자 또는 리턴값으로 사용될 수 있다.

파이썬에서 제공하는 장식자가 매우 다양하며, 사용자가 직접 장식자를 정의할 수도 있다.
여기서는 클래스에서 선언된 메서드의 종류를 구분하기 위해 사용되는 
두 개의 장식자
`@classmethod`와 `@staticmethod`를 소개한다. 

### 메서드 종류

클래스에서 선언된 메서드는 세 종류로 나뉜다.

1. 인스턴스 메서드(instance method)
    * 첫째 매개변수로 `self` 사용 (관례)
    * 클래스의 인스턴스가 생성된 후에 인스턴스와 함께 사용.
1. 클래스 메서드(class method)
    * 첫째 매개변수로 `cls` 사용 (관례)
    * 클래스 내의 모든 속성과 메서드를 `cls` 지정자와 함께 사용 가능
        단, 인스턴스 속성 사용 불가.
    * 인스턴스 없이 클래스 이름과 함께 사용
    * 해당 클래스의 모든 인스턴스에서도 사용 가능
1. 정적 메서드(static method)
    * 첫째 매개변수에 대한 의무사항 없음.
    * 클래스 내의 모든 속성과 메서드를 클래스 이름을 지정자로 사용하여 사용해야 함.
        단, 인스턴스 속성 사용 불가.
    * 인스턴스 없이 클래스 이름과 함께 사용
    * 해당 클래스의 모든 인스턴스에서도 사용 가능
    
`self`를 첫째 매개변수로 사용하는 인스턴스 메서드는 특별한 장식자가 필요 없다.
예를 들어, 
[코드 추상화: 클래스와 객체 2부](https://github.com/liganega/ProgInPython/blob/master/notebooks/PiPy08-ClassesAndInstances_Part1.ipynb)에서 
살펴본 모든 메서드는 인스턴스 메서드이다. 
반면에 클래스 메서드와 정적 메서드는 항상 장식자와 함께 선언된다.

### 클래스 메서드 장식자: `@classmethod`

클래스의 인스턴스를 생성하지 않아도 클래스 이름과 함께 사용할 수 있는 
메서드를 **클래스 메서드**(class method)라 부르며,
`@classmethod`라는 장식자와 함께 선언된다. 

```python
@classmethod
def 함수이름(cls,인자1, ..., 인자k):
    본문
```

클래스 메서드 역시 첫째 인자로 해당 클래스를 받을 준비를 하는 매개변수를 반드시 사용해야 한다.
하지만 `self` 대신에 클래스(class) 자체를 가리킨다는 의미로 `cls`를 관례적으로 사용한다. 
클래스 메서드를 호출하면 `cls`에 호출 당산자가 누구인가에 따라 
해당 클래스 이름 또는 인스턴스 이름이 자동으로 삽입된다.
즉, 클래스 메서드를 호출할 때 또한 첫째 인자는 생략한다. 

클래스 메서드를 선언할 때 사용되는 `cls` 매개변수는 클래스 자신을 가리키는 
지정자 역할을 수행한다. 
따라서 클래스 메서드 내부에서는 인스턴스 속성과 인스턴스 메서드를 활용하지 못한다. 

#### 클래스 메서드 활용법

클래스 메서드를 사용하는 이유는 크게 두 가지이다. 

첫째, 상수(constant)나 클래스 속성을 직접 활용하고자 할 때 유용하다.
이는 상수와 클래스 속성을 활용하기 위해 특정 객체가 필요하지 않기 때문이다. 
또한 경우에 따라 연관된 상수나 함수들을 하나로 클래스로 묶어서 활용할 수도 있다.
이런 경우 굳이 객체를 생성할 필요가 없다. 

In [1]:
class Title:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    @classmethod
    def allowed_titles_starting_with(cls, startswith):
        # startwith로 시작하는 타이틀 찾기
        return [t for t in cls.TITLES if t.startswith(startswith)]

    @classmethod
    def allowed_titles_ending_with(cls, endswith):
        # endswith로 끝나는 타이틀 찾기
        return [t for t in cls.TITLES if t.endswith(endswith)]


print(Title.allowed_titles_starting_with("M"))
print(Title.allowed_titles_ending_with("s"))

['Mr', 'Mrs', 'Ms']
['Mrs', 'Ms']


둘째, 리턴값으로 해당 클래스의 인스턴스를 생성하는 클래스 메서드를 사용하는 경우가 종종 있다.
이렇게 하면 해당 클래스의 인스턴스를 생성하기 위한 준비사항을 이 클래스 메서드가 알아서 처리해준다. 
예를 들어, 아래 `Person` 클래스의 `from_text_file` 메서드는 
특정 텍스트 파일에 저장된 정보를 확인한 후 그 정보를 적절히 활용하여
`Person` 클래스의 인스턴스를 생성해준다. 

In [2]:
class Person:

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @classmethod
    def fromDict(cls, nameDict):
        # {'이름': 'Jane', '성': 'Doe'} 형식의 사전 자료형에서
        # 이름과 성 정보를 추출해서 Person 클래스 객체 생성
        params = nameDict.values()
        return cls(*params)
    
janeDoe = {'이름':'Jane', '성':'Doe'}

jDoe = Person.fromDict(janeDoe)

print(jDoe.name)
print(jDoe.surname)

Jane
Doe


### 정적 메서드 장식자: `@staticmethod`

정적 메서드는 클래스나 인스터를 지정하는 인자를 사용하지 않는다.
따라서 일반 함수를 선언하는 것과 완벽하게 동일하다.
다만, 클래스 내부에서 선언되었기 때문에 항상 해당 클래스의 이름을 
지정자로 사용하여 호출된다. 

또한 정적 메서드의 본문에서 해당 클래스의 인스턴스 속성과 인스턴스 메서드는
전혀 활용되지 못한다.
해당 클래스의 클래스 메서드는 클래스의 이름을 직접 지정하여 사용해야 한다.
반면에 인스턴스 메서드는 클래스 메서드와 정적 메서드를 모두 활용할 수 있다. 

결론적으로, 클래스 메서드의 선언 및 활용 방식과 거의 동일하다. 
다만, 정적 메서드는 클래스를 지정할 때 `cls` 대신에 해당 클래스의 이름을 직접
언급해야 한다는 차이점이 있을 뿐이다.

아래 예제가 세 종류의 메서드 활용법을 잘 보여준다. 

In [3]:
class Person:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    # 인스턴스 메서드
    def fullname(self):
        return "%s %s" % (self.name, self.surname)

    # 클래스 메서드
    @classmethod
    def allowed_titles_starting_with(cls, startswith):
        return [t for t in cls.TITLES if t.startswith(startswith)]

    # 정적 메서드
    @staticmethod
    def allowed_titles_ending_with(endswith):
        return [t for t in Person.TITLES if t.endswith(endswith)]


jane = Person("Jane", "Smith")

print(jane.fullname())

print(jane.allowed_titles_starting_with("M"))
print(Person.allowed_titles_starting_with("M"))

print(jane.allowed_titles_ending_with("s"))
print(Person.allowed_titles_ending_with("s"))

Jane Smith
['Mr', 'Mrs', 'Ms']
['Mr', 'Mrs', 'Ms']
['Mrs', 'Ms']
['Mrs', 'Ms']


### 연습 4

1. 다음 속성과 메서드를 포함하는 클래스 `Numbers`를 정의하라.  
    * `MULTIPLIER`: 클래스 속성
    * `__init__` 메서드: 숫자 두 개를 입력받아 각각 인스턴스 속성 `x`와 `y`로 저장.
    1. `add`: 인스턴스 메서드. `x`와 `y`의 합 내주기
    1. `multiply`: 클래스 메서드. 하나의 숫자 `a`를 입력 받아 `MULTIPLIER`와 곱센 결과 내주기
    1. `subtract`: 정적 메서드. `b`와 `c` 숫자 두 개를 입력 받아 `b-c` 내주기

In [4]:
class Numbers:
    MULTIPLIER = 3.5

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def add(self):
        return self.x + self.y

    @classmethod
    def multiply(cls, a):
        return cls.MULTIPLIER * a

    @staticmethod
    def subtract(b, c):
        return b - c

In [5]:
twoAndfive = Numbers(2,5)

print(twoAndfive.add())

print(Numbers.multiply(4))
print(twoAndfive.multiply(4))

print(Numbers.subtract(7, 2))
print(twoAndfive.subtract(7,2))

7
14.0
14.0
5
5


## 매직 메서드

## 매직 메서드 재정의